In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import shutil
import os
import logging
import sys
sys.path.append('..')

from textblob import TextBlob
from pprint import pprint
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from transformers import AutoModelForSequenceClassification

from finbert.finbert import *
import finbert.utils as tools
from finbert.custom_finbert import CustomFinBert

project_dir = Path.cwd().parent
pd.set_option('max_colwidth', -1)

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.ERROR)

In [2]:
standard_dir = project_dir.parent.parent

# ! TODO: PUT THE PREPROCESSING CODE HERE AND STORE PROCESSED DF

# Prepare model

In [3]:
lm_path = project_dir/'models'/'language_model'/'finbertTRC2'
final_model_dir = project_dir/'models'/'classification_model_256_hidden_dim'
data_dir = standard_dir/"Data"
# classification_data_path= standard_dir/'Data'/'sentiment_data'

## Configure training parameters

In [4]:
# Clean the cl_path
try:
    shutil.rmtree(final_model_dir) 
except:
    pass

## load the pretrained language model trained on Reuters TRC2
bertmodel = AutoModelForSequenceClassification.from_pretrained(
    lm_path, cache_dir=None, num_labels=30)

Some weights of the model checkpoint at /media/student/HDD 2/Michelle/MLS/advanced/finBERT_transaction_classification/models/language_model/finbertTRC2 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
S

In [5]:
custom_finbert = CustomFinBert(bertmodel, 38, 30, hidden_dims=[256])

In [6]:
config = Config(
    data_dir=data_dir,
    model_dir=final_model_dir,
    bert_model=custom_finbert,
    num_train_epochs=10,
    max_seq_length=48,
    train_batch_size=32,
    learning_rate=2e-5,
    output_mode="classification",
    warm_up_proportion=0.2,
    local_rank=-1,
    discriminate=True,
    gradual_unfreeze=False,
    numeric_params=38,
    label_colname="category"
)

In [7]:
finbert = FinBert(config, transaction_classification=True)

In [8]:
y = pd.read_csv(standard_dir/"Data"/"eval.csv")["category"]

In [9]:
finbert.prepare_model(label_list=y.unique().tolist())

07/18/2024 11:13:47 - INFO - finbert.finbert -   device: cuda n_gpu: 2, distributed training: False, 16-bits training: False


In [10]:
train_examples, numeric_feats = finbert.get_data("train")

In [11]:
model = finbert.create_the_model()

In [12]:
trained_model = finbert.train(train_examples = train_examples, model = model, train_numeric_feats=numeric_feats)

07/18/2024 11:13:56 - INFO - finbert.utils -   *** Example ***
07/18/2024 11:13:56 - INFO - finbert.utils -   guid: 0
07/18/2024 11:13:56 - INFO - finbert.utils -   tokens: [CLS] ac ##h withdrawal am ##mar ##yse hem ##ant ##ne ##x ##press transfer [SEP]
07/18/2024 11:13:56 - INFO - finbert.utils -   input_ids: 101 9353 2232 10534 2572 7849 23274 19610 4630 2638 2595 20110 4651 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:13:56 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:13:56 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:13:56 - INFO - finbert.utils -   label: Transfer Debit (id = 2)
07/18/2024 11:14:08 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 11:14:08 - INFO - finbert.finbert -     Num examples = 160391
07/18

Iteration:   0%|          | 0/5013 [00:00<?, ?it/s]

(Iteration 0) Training loss: 3.5488927364349365
(Iteration 100) Training loss: 3.3954172134399414
(Iteration 200) Training loss: 3.4377686977386475
(Iteration 300) Training loss: 3.3448245525360107
(Iteration 400) Training loss: 3.4722142219543457
(Iteration 500) Training loss: 3.2703239917755127
(Iteration 600) Training loss: 3.3134829998016357
(Iteration 700) Training loss: 3.195587158203125
(Iteration 800) Training loss: 3.125565528869629
(Iteration 900) Training loss: 3.1242454051971436
(Iteration 1000) Training loss: 3.1316120624542236
(Iteration 1100) Training loss: 3.1121156215667725
(Iteration 1200) Training loss: 2.914438247680664
(Iteration 1300) Training loss: 2.8431332111358643
(Iteration 1400) Training loss: 2.9618353843688965
(Iteration 1500) Training loss: 2.888730049133301
(Iteration 1600) Training loss: 2.765958547592163
(Iteration 1700) Training loss: 2.582751512527466
(Iteration 1800) Training loss: 2.3954269886016846
(Iteration 1900) Training loss: 2.292456865310669

07/18/2024 11:34:10 - INFO - finbert.utils -   *** Example ***
07/18/2024 11:34:10 - INFO - finbert.utils -   guid: 0
07/18/2024 11:34:10 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 11:34:10 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:34:10 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:34:10 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 11:34:10 - INFO - finbert.utils -   label: Loans (id = 0)
07/18/2024 11:34:15 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 11:34:15 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 11:34:15 - INFO - fin

Validating:   0%|          | 0/2149 [00:00<?, ?it/s]

...

(Iteration 0) Training loss: 0.06415817141532898
(Iteration 100) Training loss: 0.06305783241987228
(Iteration 200) Training loss: 0.5804417729377747
(Iteration 300) Training loss: 0.20280034840106964
(Iteration 400) Training loss: 0.40526047348976135
(Iteration 500) Training loss: 0.188215970993042
(Iteration 600) Training loss: 0.3974580466747284
(Iteration 700) Training loss: 0.10390491038560867
(Iteration 800) Training loss: 0.13635551929473877
(Iteration 900) Training loss: 0.08073074370622635
(Iteration 1000) Training loss: 0.06034556031227112
(Iteration 1100) Training loss: 0.07767575234174728
(Iteration 1200) Training loss: 0.13206328451633453
(Iteration 1300) Training loss: 0.09452766925096512
(Iteration 1400) Training loss: 0.06325747072696686
(Iteration 1500) Training loss: 0.08373218774795532
(Iteration 1600) Training loss: 0.049672681838274
(Iteration 1700) Training loss: 0.10852455347776413
(Iteration 1800) Training loss: 0.05470076948404312
(Iteration 1900) Training loss

07/18/2024 14:58:37 - INFO - finbert.utils -   *** Example ***
07/18/2024 14:58:37 - INFO - finbert.utils -   guid: 0
07/18/2024 14:58:37 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 14:58:37 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 14:58:37 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 14:58:37 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 14:58:37 - INFO - finbert.utils -   label: Loans (id = 0)
07/18/2024 14:58:42 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 14:58:42 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 14:58:42 - INFO - fin

Validating:   0%|          | 0/2149 [00:00<?, ?it/s]

Validation losses: [1.0823279039673386, 0.567875035701726, 0.4626183950135518, 0.4138067854482121, 0.4124975941745671, 0.4031076118135075, 0.38278619712868706, 0.3806190205803063, 0.3842699571994552, 0.3820825545351281]
Validation Accuracy: 0.9020


Epoch: 100%|██████████| 10/10 [3:46:57<00:00, 1361.76s/it]


In [13]:
import pickle
with open("./trained_model_3.pkl", "wb") as f:
    pickle.dump(trained_model, f)

In [29]:
np.argmax(trained_model.cpu().detach().numpy(), -1).shape

(128,)

In [14]:
test_examples, test_numeric_feats = finbert.get_data("validation")

In [15]:
results = finbert.evaluate(trained_model, test_examples, test_numeric_feats)

07/18/2024 15:14:02 - INFO - finbert.utils -   *** Example ***
07/18/2024 15:14:02 - INFO - finbert.utils -   guid: 0
07/18/2024 15:14:02 - INFO - finbert.utils -   tokens: [CLS] rt ##p received money ##lion ins ##ta ##cas ##h 48 ##8 [SEP]
07/18/2024 15:14:02 - INFO - finbert.utils -   input_ids: 101 19387 2361 2363 2769 18964 16021 2696 15671 2232 4466 2620 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:14:02 - INFO - finbert.utils -   attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:14:02 - INFO - finbert.utils -   token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
07/18/2024 15:14:02 - INFO - finbert.utils -   label: Loans (id = 0)


07/18/2024 15:14:07 - INFO - finbert.finbert -   ***** Loading data *****
07/18/2024 15:14:07 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 15:14:07 - INFO - finbert.finbert -     Batch size = 32
07/18/2024 15:14:07 - INFO - finbert.finbert -     Num steps = 21480
07/18/2024 15:14:07 - INFO - finbert.finbert -   ***** Running evaluation ***** 
07/18/2024 15:14:07 - INFO - finbert.finbert -     Num examples = 68739
07/18/2024 15:14:07 - INFO - finbert.finbert -     Batch size = 32


Testing:   0%|          | 0/2149 [00:00<?, ?it/s]

Evaluation loss: 0.2722044852638089


In [16]:
preds = results["predictions"].apply(lambda x: np.argmax(x, axis=0))

In [17]:
label_mapping = {i:label for i, label in enumerate(finbert.label_list)}

In [18]:
preds = [label_mapping[p] for p in preds]

In [19]:
labels = [label_mapping[l] for l in results["labels"]]

In [20]:
print(classification_report(labels, preds, digits=3))

/home/student/anaconda3/envs/mls/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                            precision    recall  f1-score   support

                       ATM      0.995     0.999     0.997      1702
    Arts and Entertainment      0.872     0.975     0.921       119
                 Bank Fees      0.993     0.992     0.992      1940
             Check Deposit      0.939     0.984     0.961        63
  Clothing and Accessories      0.823     0.890     0.855       957
        Convenience Stores      0.713     0.694     0.703      5589
         Department Stores      0.636     0.842     0.724       601
     Digital Entertainment      0.928     0.936     0.932      1358
Food and Beverage Services      0.690     0.777     0.731       103
              Gas Stations      0.459     0.584     0.514      3876
  Gyms and Fitness Centers      0.633     0.905     0.745        21
                Healthcare      0.629     0.710     0.667        62
                 Insurance      0.955     0.892     0.922       526
                  Interest      1.000     1.000